In [1]:
!pip install -r requirements.txt

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import dataset_loader as loader
from pprint import PrettyPrinter
from tqdm.auto import tqdm
pprint = PrettyPrinter(indent=2).pprint
from datetime import datetime
# import tensorflow_addons as tfa

tf.__version__:2.3.1


In [4]:
import matplotlib.pyplot as plt

In [5]:
from dotmap import DotMap

import tensorflow as tf
# import data_utils 
import numpy as np
import os
from pathlib import Path
from functools import partial
import audio_transform

# Config

In [6]:
model_config = DotMap({
  "conv_type":"conv1d",
  "depth":5,
  "filters":256,
  "global_kernel_size":4,
  "use_dilate":True,
  "drop_rate":.3,
  "concat_mode":"None",
  "gaussian_drop_rate":.05,
  "activation":tf.keras.activations.relu,
#   "final_group_mode": ["attention", "last"],
  "final_group_mode": ["attention", "last"],
})

other_config = DotMap({
  "lr":1e-4,
  "use_remark":False,
  "multilabel_problem":True,
  "total_folds":12,
  "epoches":1000
})

big_config = DotMap({
  "model_config":model_config,
  "other_config":other_config,
})

In [7]:
try:
  get_ipython
  random_flag = False
except:
  random_flag = True
random_flag

False

In [8]:
# if random_flag:
#   model_meta.depth = int(np.random.randint(4, 12))
#   model_meta.filters = np.random.choice(
#     [
#       16,
#       32,
#       64,
#       128,
# #       256,
# #       512,
#     ]
#   )
#   model_meta.conv_type = np.random.choice(["conv2d", "conv1d"])
#   model_meta.kernel_size = int(np.random.randint(2,9))
#   model_meta.drop_rate = np.random.choice([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5])
#   model_meta.use_dilate = np.random.choice([True, False])
#   model_meta.concat_mode = np.random.choice(["None", "last", "all", "residual"])
#   model_meta.lr = np.random.choice([1/10**power for power in range(2,6)])
#   model_meta.gaussian_drop_rate = np.random.choice([0,0.05,.1,0.15,.2,0.25,.3])
#   model_meta.activation=np.random.choice(
#     [
#       tf.keras.activations.relu,
#       tf.keras.activations.tanh,
#       tf.keras.activations.selu,
#       tf.keras.activations.sigmoid,
#     ]
#   )
#   model_meta.final_group_mode = np.random.choice([
#     ["mean"],
#     ["max"],
#     ["mean", 'max'],
#     ["attention","last"],
#     ["attention","first_and_last"],
#     ["attention", "all"],
#     ["attention", "all", "max"]
#   ])
# #   model_meta.use_remark = np.random.choice([
# #     True, False
# #   ])
#   model_meta.multilabel_problem = np.random.choice([True,False])

pprint(big_config)
pprint(big_config.model_config.activation.__name__)

{ 'model_config': { 'activation': <function relu at 0x7f81652cd680>,
                    'concat_mode': 'None',
                    'conv_type': 'conv1d',
                    'depth': 5,
                    'drop_rate': 0.3,
                    'filters': 256,
                    'final_group_mode': ['attention', 'last'],
                    'gaussian_drop_rate': 0.05,
                    'global_kernel_size': 4,
                    'use_dilate': True},
  'other_config': { 'epoches': 1000,
                    'lr': 0.0001,
                    'multilabel_problem': True,
                    'total_folds': 12,
                    'use_remark': False}}
'relu'


# Model

In [9]:
def my_dropout(x, drop_rate, gaussian_drop_rate):
  x = tf.keras.layers.GaussianDropout(gaussian_drop_rate)(x)
  LEN = len(x.shape)
  power = 1/LEN
  each_drop_rate = 1-(1-drop_rate)**power
  for channel in range(1,LEN):
    noise_shape = [None]*LEN
    noise_shape[channel] = 1
    x = tf.keras.layers.Dropout(each_drop_rate,noise_shape=noise_shape)(x)
  x = tf.keras.layers.Dropout(each_drop_rate,noise_shape=[None]*LEN)(x)
  return x

In [10]:
def conv_block_2d(x,filters,kernel_size,drop_rate, gaussian_drop_rate,activation,**conv_kwargs):
  assert len(x.shape) == 4, f"only support dims==4 for conv2d, got {len(x.shape)}"
  conv_on_freq = tf.keras.layers.Conv2D(
    1,
    (1,kernel_size),
    padding="same"
  )
  conv_on_time = tf.keras.layers.Conv2D(
    filters,
    (kernel_size,1),
    padding="same",
    **conv_kwargs
  ) 
  x = my_dropout(x, drop_rate,gaussian_drop_rate)
  x = conv_on_freq(x)
  x = activation(x)
  x = my_dropout(x, drop_rate,gaussian_drop_rate)
  x = conv_on_time(x)
  x = activation(x)
  
  return x

def conv_block_1d(x,filters,kernel_size,drop_rate, gaussian_drop_rate,activation,**conv_kwargs):
  assert len(x.shape) == 3, f"only support dims==3 for conv1d, got {len(x.shape)}"
  x = my_dropout(x, drop_rate,gaussian_drop_rate)
  conv = tf.keras.layers.Conv1D(
    filters, kernel_size,
    padding="same", **conv_kwargs)
  x = conv(x)
  x = activation(x)
  return x

In [11]:
if other_config.use_remark:
  categories=9
else:
  categories=6
def build_model(
  depth, filters,
  use_dilate,
  activation,
  global_kernel_size,
  drop_rate,
  gaussian_drop_rate,
  conv_type,
  concat_mode,
  final_group_mode,
  input_shape=(216, 128),
):

  layer_params = [
    [filters, [1, [2**power,]][use_dilate]] for power in range(depth)
  ]
  
  batch_shape = [None, *input_shape]
  if conv_type == "conv2d":
    batch_shape += [1]
    for p in layer_params:
      p[-1].append(1)
  elif conv_type == "conv1d":
    pass
  else:
    raise ValueError(f"conv_type cannot be {conv_type}")
  conv_block = {
    "conv2d":conv_block_2d,
    "conv1d":conv_block_1d,
  }[conv_type]
  
  Input = tf.keras.layers.Input(batch_shape=batch_shape)
  nodes = [Input]
  
  for filters, dilation_rate in layer_params:
    if concat_mode in ["None", None]:
      inputs = nodes[-1]
    elif concat_mode == "last":
      inputs = tf.concat(nodes[-2:], axis=-1)
    elif concat_mode == "all":
      inputs = tf.concat(nodes, axis=-1)
    elif concat_mode == "residual":
      if len(nodes) >= 2 and nodes[-1].shape[-1] == nodes[-2].shape[-1]:
        inputs = nodes[-1]+nodes[-2]
      else:
        inputs = nodes[-1]
    else:
      raise
    
    nodes.append(
      conv_block(
        inputs,
        filters,
        global_kernel_size,
        drop_rate=drop_rate,
        gaussian_drop_rate=gaussian_drop_rate,
        activation=activation,
        dilation_rate=dilation_rate,
      )
    )
  
  for hidden in [categories]:
    nodes.append(
      tf.keras.layers.Dense(
        hidden,
      )(nodes[-1])
    )
  
  final_ = []
  if "max" in final_group_mode:
    final_.append(tf.keras.layers.GlobalMaxPool1D()(nodes[-1]))
  if "mean" in final_group_mode:
    final_.append(tf.keras.layers.GlobalAveragePooling1D()(
      tf.keras.layers.Dropout(drop_rate)(nodes[-1])
    ))
  if "attention" in final_group_mode:
    if "last" in final_group_mode:
      inputs = nodes[-2] # before to categories
    elif "all" in final_group_mode:
      inputs = tf.concat(nodes, axis=-1)
    elif "first_and_last" in final_group_mode:
      inputs = tf.concat([nodes[0], nodes[-2]], axis=-1)
    
    hidden_neurons = filters
    before_porb_hidden = tf.keras.layers.Dense(hidden_neurons, activation="relu")(inputs)
    before_prob = tf.keras.layers.Dense(1, activation="relu")(before_porb_hidden/(inputs.shape[-1]**.5))
    prob = tf.keras.layers.Softmax(axis=-2)(before_prob)
    final_.append(
      tf.reduce_sum(
        tf.keras.layers.Dropout(drop_rate)(prob*nodes[-1]), axis=-2
      )
    )
    
  nodes.append(tf.reduce_mean(final_,axis=0))
  
#   print(nodes)
  model = tf.keras.Model(
    inputs=nodes[0],
    outputs=nodes[-1]
  )
  
  return model


def get_initial_model(**model_config):
  model = build_model(
    **model_config
  )
  return model


In [12]:
model = get_initial_model(**big_config.model_config)

In [13]:
model.summary()
del model

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 216, 128)]   0                                            
__________________________________________________________________________________________________
gaussian_dropout (GaussianDropo (None, 216, 128)     0           input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 216, 128)     0           gaussian_dropout[0][0]           
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 216, 128)     0           dropout[0][0]                    
_______________________________________________________________________________________

# Before Training

In [14]:
loader.config_helper()

def config_helper():
  print(inspect.getsource(config_helper))
  print("="*30)
  print(inspect.getsource(load_default_config))

def load_default_config():
  config = {
    "seed":228922,
    "total_folds":12,
    "cache":True,
    "use_shifted_mel":True,
    "use_remark":False,
    "shift_mel_kwargs":{"n":5},
    "batch_size_top":32,
    "batch_size_low":1,
  }
  return config



In [15]:
ds_folds_list, ds_config, base_ds = loader.lazy_ds_loader(
  use_shifted_mel=False,
  total_folds=other_config.total_folds,
  use_remark=other_config.use_remark,
  batch_size_low=32,
)

In [16]:
ds_config

{'seed': 228922,
 'total_folds': 12,
 'cache': True,
 'use_shifted_mel': False,
 'use_remark': False,
 'shift_mel_kwargs': {'n': 5},
 'batch_size_top': 32,
 'batch_size_low': 32}

In [17]:
D = {}
if ds_config["cache"]:
  print("initital cached ds")
  for _ in tqdm(base_ds):
    rrr=_[-1].numpy()
    if rrr not in D:
      D[rrr]=0
    D[rrr]+=1
    continue

D

initital cached ds


{b'Barking': 200,
 b'Crying': 200,
 b'Howling': 200,
 b'GlassBreaking': 199,
 b'COSmoke': 200,
 b'Vacuum': 89,
 b'Dishes': 60,
 b'Cat': 30,
 b'Blender': 10,
 b'Electrics': 10,
 b'Other': 1}

In [18]:
def make_input_right(*args):
  assert len(args)==2
  a0,a1 = args
  if model_config.conv_type == "conv1d":
    pass
  elif model_config.conv_type == "conv2d":
    a0 = args[0][..., tf.newaxis]
  else:
    raise
  
  a1 = tf.one_hot(a1.numpy().astype(np.int64), categories)
  
  return a0, a1

In [19]:
model_folder = "TEST_MODEL_0608_attention"

In [20]:
from pathlib import Path
Path(model_folder).mkdir(exist_ok=True,parents=True)

In [21]:
from collections import defaultdict
import json

In [22]:
json.dump({k:str(v) for k,v in dict(model_config).items()}, open(model_folder+"/model_config.json","w"))
json.dump({k:str(v) for k,v in dict(other_config).items()}, open(model_folder+"/other_config.json","w"))

In [23]:
def mix_up_on(ds):
  for _1, _2 in zip(ds, ds):
    r = np.random.beta(0.1, 0.1,tf.shape(_1[0])[0]).astype(np.float32)
    r0 = tf.reshape(r, [-1,1,1])
    r1 = tf.reshape(r, [-1,1])
    
    yield _1[0]*r0+_2[0]*(1-r0), _1[1]*r1+_2[1]*(1-r1)

In [ ]:
for ith_fold, (train_ds, valid_ds) in enumerate(ds_folds_list):
  train_ds = train_ds.map(
    lambda x,y: tf.py_function(
      func=make_input_right,
      inp=[x,y],
      Tout=(tf.float32,tf.float32)
    )
  )
  
  valid_ds = valid_ds.map(
    lambda x,y : tf.py_function(func=make_input_right,
      inp=[x,y],
      Tout=(tf.float32,tf.float32)
    )
  )
  
  opt = tf.keras.optimizers.Adam(other_config.lr)
  if other_config.multilabel_problem:
    loss = tf.keras.losses.BinaryCrossentropy(
      from_logits=True,
      name="loss",
    )
    metrics = [tf.keras.metrics.BinaryAccuracy(name="bin_acc")]
  else:
    loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True,
      name="loss",
    )
    metrics = tf.keras.metrics.CategoricalAccuracy(
      name="acc"
    )
  
  model = get_initial_model(**big_config.model_config)
  model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metrics,
  )
  
  patient = 100
  monitor="loss"
  
  base_model_path = model_folder+f"/model.valid_{ith_fold:02d}"
  last_model_path = base_model_path+".last.h5"
  best_val_model_path = base_model_path+".best_val.h5"
  best_train_model_path = base_model_path+".best_train.h5"
  history_path = model_folder+f"/history.valid_{ith_fold:02d}.json"
  
  cbs = [
    # best_val_model
    tf.keras.callbacks.ModelCheckpoint(
      filepath=best_val_model_path, monitor="val_"+monitor, verbose=1, save_best_only=True,
    ),
    # best_val_model
    tf.keras.callbacks.ModelCheckpoint(
      filepath=best_train_model_path, monitor=monitor, verbose=1, save_best_only=True,
    ),
    # last model
    tf.keras.callbacks.ModelCheckpoint(
      filepath=best_train_model_path, verbose=0,
    ),
    # early_stop with val 
    tf.keras.callbacks.EarlyStopping(
      monitor="val_"+monitor, patience=patient, verbose=1,
    )
  ]
  
  mix_uped_train = tf.data.Dataset.from_generator(
    partial(mix_up_on,train_ds),
    (tf.float32, tf.float32)
  )
  all_history = defaultdict(list)
  print(datetime.time(datetime.now()))
  history = model.fit(
#     train_ds.prefetch(5),
    mix_uped_train.prefetch(5),
    epochs=other_config.epoches,
    validation_data=valid_ds.prefetch(5),
    verbose=2,
    callbacks=cbs
  )
  for k, v in history.history.items():
    all_history[k]+=v
    
  keys, lists = zip(*all_history.items())
  best_ith_epoch = np.argmin(all_history["val_"+monitor])
  print(f"best_epoch : {best_ith_epoch}")
  print(keys, ", (", *map(lambda l:f"{l[best_ith_epoch]:.3f}", lists), ")")
    
  with open(history_path,"w") as f:
    json.dump(all_history,f)

01:43:09.615757
Epoch 1/1000

Epoch 00001: val_loss improved from inf to 0.60308, saving model to TEST_MODEL_0608_attention/model.valid_00.best_val.h5

Epoch 00001: loss improved from inf to 1.28998, saving model to TEST_MODEL_0608_attention/model.valid_00.best_train.h5
32/32 - 2s - loss: 1.2900 - bin_acc: 0.6485 - val_loss: 0.6031 - val_bin_acc: 0.8249
Epoch 2/1000

Epoch 00002: val_loss improved from 0.60308 to 0.54722, saving model to TEST_MODEL_0608_attention/model.valid_00.best_val.h5

Epoch 00002: loss improved from 1.28998 to 0.47688, saving model to TEST_MODEL_0608_attention/model.valid_00.best_train.h5
32/32 - 1s - loss: 0.4769 - bin_acc: 0.7018 - val_loss: 0.5472 - val_bin_acc: 0.8451
Epoch 3/1000

Epoch 00003: val_loss improved from 0.54722 to 0.53372, saving model to TEST_MODEL_0608_attention/model.valid_00.best_val.h5

Epoch 00003: loss improved from 0.47688 to 0.44588, saving model to TEST_MODEL_0608_attention/model.valid_00.best_train.h5
32/32 - 2s - loss: 0.4459 - bin_a